In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adam
from unet import UNet
from preprocessing_1_dataloader import get_data

In [2]:
####Initialisation####
#create 2 network
Student = UNet(3,2)
Teacher = UNet(3,2)
#creat the losses
sup_crit = nn.CrossEntropyLoss()
unsup_crit = nn.CrossEntropyLoss()
#optimizer
optimizer = Adam(Student.parameters())
#other HP
batch_size = 64
epochs = 16
ramp_up = 10
consistency = 56
alpha = 0.999
gs = 0

In [3]:
#Weigth coef for the Unsupervised
def wt(rampup_length, current, alpha):
    if rampup_length == 0:
                return 1.0
    else:
        current = np.clip(current, 0.0, rampup_length)
        phase = 1.0 - current / rampup_length
        return float(alpha * np.exp(-5.0 * phase * phase))
#update the Teacher weigth
def update_ema_variables(model, ema_model, alpha, global_step): 
    # Use the true average until the exponential average is more correct
    alpha = min(1 - 1 / (global_step + 1), alpha)
    for ema_param, param in zip(ema_model.parameters(), model.parameters()):
        ema_param.data.mul_(alpha).add_(1 - alpha, param.data)

In [4]:
##data loader
mixed_train_loader, val_loader, test_loader = get_data(0.2,0.8,0.2,0.1)

FileNotFoundError: [Errno 2] No such file or directory: './data/images'

In [ ]:
for epoch in range(epochs):
    cum_loss = 0
    for idx, (X,y) in enumerate(mixed_train_loader):
        optimizer.zero_grad()
        pred_stud = Student(X)
        pred_teach = Teacher(X)

        # Find img with label
        idx = [elem != -1 for elem in y[:, 0, 0, 0]] #If batchsize is the first dim

        # Calculate supervised and unsupervised losses
        Ls = sup_crit(pred_stud[idx], y[idx])
        Lu = unsup_crit(pred_stud, pred_teach)
        loss = Ls + Lu * wt(ramp_up, epoch, consistency)
        loss.backward()
        optimizer.step()
        cum_loss += loss
        gs += 1
        update_ema_variables(Student, Teacher, alpha, gs)
    print(f'Epoch {epoch} loss {cum_loss.item() / len(trainloader)}')